In [6]:
import geopandas
import pystac_client
import odc.stac
import rioxarray
import rasterio
from rasterio.transform import from_bounds

# Load the study area
study_area = geopandas.read_file('study_area.gpkg')
bbox = study_area.total_bounds

# Define STAC API URL and collection ID
api_url = "https://earth-search.aws.element84.com/v1"
collection_id = "sentinel-2-l2a"

# Open the STAC API client
client = pystac_client.Client.open(api_url)

# Dates to process
dates = ["2017-04-08", "2017-04-28", "2017-07-27"]

# Specify the CRS and resolution manually
crs = 'EPSG:32755'
resolution = 20  # in meters, adjust as necessary

for date in dates:
    # Search for items for the specific date
    search = client.search(
        collections=[collection_id],
        datetime=date,
        bbox=bbox
    )

    item_collection = search.item_collection()

    # Check if any items are found for the date
    if not item_collection:
        print(f"No items found for date {date}.")
        continue

    # Load the dataset using ODC STAC
    ds = odc.stac.load(
        item_collection,
        groupby='solar_day',
        chunks={'x': 2048, 'y': 2048},
        use_overviews=True,
        crs=crs,
        resolution=resolution,
        bbox=study_area.total_bounds,
        bands=[ 'blue', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22']
    )

    # Extract spatial information using rioxarray from one band
    example_band = ds.red.isel(time=0)
    example_band = example_band.rio.write_crs(crs)
    bounds = example_band.rio.bounds()
    height, width = example_band.shape

    # Define the transform and metadata for the TIFF file
    transform = from_bounds(*bounds, width, height)

    # Save each band as a separate TIFF file for the specific date
    for band_name in ds.data_vars:
        band_data = ds[band_name].isel(time=0)
        band_data = band_data.rio.write_crs(crs)  # Write CRS to each band

        # Create a filename that includes the band name and date
        filename = f'{band_name}_{date.replace("-", "")}.tif'
        
        # Save the band as a TIFF file
        with rasterio.open(
            filename, 'w', driver='COG',
            height=height, width=width,
            count=1, dtype=band_data.dtype,
            crs=crs, transform=transform,
        ) as dst:
            dst.write(band_data.values, 1)

    print(f"All bands for {date} saved as separate TIFF files.")


All bands for 2017-04-08 saved as separate TIFF files.
All bands for 2017-04-28 saved as separate TIFF files.
All bands for 2017-07-27 saved as separate TIFF files.


In [16]:
import rasterio
import numpy as np

dates = ["20170408", "20170428", "20170727"]

for date in dates:


    red_tif = 'red_'+date+'.tif'
    green_tif = 'green_'+date+'.tif'
    blue_tif = 'blue_'+date+'.tif'
    output_tif = 'visual_'+date+'.tif'
    
    # Open the red, green, and blue GeoTIFF files
    with rasterio.open(red_tif) as src_red:
        red = src_red.read(1)
        profile = src_red.profile

    with rasterio.open(green_tif) as src_green:
        green = src_green.read(1)

    with rasterio.open(blue_tif) as src_blue:
        blue = src_blue.read(1)

    # Stack the bands together to form an RGB image
    rgb = np.stack((red, green, blue), axis=0)

    # Update the profile for a 3-band image
    profile.update(count=3)

    # Write the RGB image to a new GeoTIFF file
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(rgb[0], 1)  # Red channel
        dst.write(rgb[1], 2)  # Green channel
        dst.write(rgb[2], 3)  # Blue channel